# E-commerce Sales Analysis Pipeline
**Objective:** Clean and analyze a large-scale retail dataset to extract business KPIs.<br>
**Author:** [Rafael Campos Andrés]<br>
**Date:** January 2026

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

In [ ]:
# Loading raw data
base_dir = Path.cwd().parent
data_path = base_dir / "data" / "raw_sales_data.csv"
if data_path.exists():
    try:
        df = pd.read_csv(data_path)
        df['date'] = pd.to_datetime(df['date'])
        print(f"File: {data_path} uploaded succesfully")
    except:
        print(f"File: {data_path} is empty or could not be found")
else:
    print(f"File: {data_path} could not be found")

In [ ]:
# Cleaning category names
df['category'] = df['category'].str.replace('_', '').str.strip().str.capitalize()
condition = df['category'].str.contains('Elec', na=False)
df['category'] = np.where(condition, 'Electronics', df['category'])
print(df.sample(5))

In [ ]:
# Fill na
df['customer_email'] = df['customer_email'].fillna('user@gmail.com')

df['unit_price'] = df.groupby('product')['unit_price'].transform('mean')
print(df.sample(5))

In [ ]:
# Profit analysis
df['revenue'] = df['quantity'] * df['unit_price']

days_map = {
    0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 
    4: 'Fri', 5: 'Sat', 6: 'Sun'
}
months_map = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
temp = df[['date', 'revenue']].copy()
temp['day'] = temp['date'].dt.day_of_week
temp['month'] = temp['date'].dt.month
day_of_week_analysis = temp.groupby('day')['revenue'].mean().sort_index()
month_analysis = temp.groupby('month')['revenue'].mean().sort_index()
day_of_week_analysis.index = day_of_week_analysis.index.map(days_map)
month_analysis.index = month_analysis.index.map(months_map)

mean_week = day_of_week_analysis.mean()
mean_year = month_analysis.mean()

plt.figure(figsize=(4, 2))
sns.lineplot(x=day_of_week_analysis.index, y=day_of_week_analysis.values, marker='o')
plt.axhline(mean_week, color='red', linestyle='--', label=f'Mean: {mean_week:.2f}')
plt.title('Average Revenue by Day')
plt.legend()
plt.show()

plt.figure(figsize=(8, 2))
sns.lineplot(x=month_analysis.index, y=month_analysis.values, marker='o')
plt.axhline(mean_year, color='red', linestyle='--', label=f'Mean: {mean_year:.2f}')
plt.title('Average Revenue by Month')
plt.legend()
plt.show()

In [ ]:
# Analyze profits per product
product_analysis = df.copy()
product_analysis = product_analysis.groupby('product').agg(
    units_sold = ('quantity', 'sum'),
    mean_price = ('unit_price', 'mean'),
    real_millions = ('revenue', 'sum')
).sort_values(by='real_millions', ascending=False)
product_analysis['expected_millions'] = (product_analysis['mean_price'] * product_analysis['units_sold']) / 1000000
product_analysis['real_millions'] = product_analysis['real_millions'] / 1000000
product_analysis['reconciliation'] = product_analysis['real_millions'] - product_analysis['expected_millions']
print(product_analysis)

plt.figure(figsize=(5, 3))
plt.title("Product Profits Comparison")
sns.barplot(data=product_analysis, x='product', y='real_millions').set(xlabel=None, ylabel="Millions")
plt.axhline(y=product_analysis['real_millions'].mean(), color='red', linestyle='--', label='Mean')
plt.legend()

min_val = product_analysis['real_millions'].min() * 0.95 
max_val = product_analysis['real_millions'].max() * 1.05
plt.ylim(min_val, max_val)

plt.show()

In [ ]:
# Compare sales quantity and revenue by category between citys
city_analysis = df.copy()
city_analysis = city_analysis.groupby(['city', 'category']).agg(
    sales = ('order_id', 'count'),
    profit = ('revenue', 'sum')
)
city_analysis['profit'] = (city_analysis['profit'] / 1000000)
print(city_analysis)

plt.figure(figsize=(8, 4))
sns.barplot(data=city_analysis, x='city', y='profit', hue='category').set(xlabel=None, ylabel='Millions')

min_val = city_analysis['profit'].min() * 0.95
max_val = city_analysis['profit'].max() * 1.05
plt.ylim(min_val, max_val)
plt.legend()
plt.show()

plt.figure(figsize=(8, 4))
sns.barplot(data=city_analysis, x='city', y='sales', hue='category').set(xlabel=None, ylabel='Sales')

min_val = city_analysis['sales'].min() * 0.95
max_val = city_analysis['sales'].max() * 1.05
plt.ylim(min_val, max_val)
plt.legend()
plt.show()

In [ ]:
# Customer revenue distribution
customer_analysis = df.copy()

conditions = [
    customer_analysis['customer_email'].str.startswith('client'),
    customer_analysis['customer_email'].str.startswith('user'),
    customer_analysis['customer_email'].str.contains('admin')
]

types = ['client', 'user', 'admin']

customer_analysis['user_type'] = np.select(conditions, types, default='user')
customer_analysis = customer_analysis.groupby('user_type').agg(
    orders = ("order_id", "count"),
    spent = ("revenue", "sum")
).sort_values(by="spent", ascending=False)
print(customer_analysis)

plt.figure(figsize=(3, 3))
plt.pie(customer_analysis['spent'], 
        labels=customer_analysis.index, 
        autopct='%1.1f%%',
        pctdistance=0.77,
        startangle=90
        )
plt.title("Total Revenue Distribution")

centre_circle = plt.Circle((0,0), 0.50, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.show()